# Elman

Elman网络大致可分为四层：输入层、隐藏层、承接层和输出层

* 输入层：接受信号输入
* 隐藏层：线性以及非线性变换
* 输出层：加权输出
* 承接层：承接上一时刻的信息

相较于简单的BP网络，Elman网络具有承接层，从而对序列数据具有记忆能力，实现动态建模

## 符号定义

|符号|含义|
|:-:|:-:|
|$\bm{x(t)}$|t时刻的输入信号|
|$\bm{h}$|隐藏层输出|
|$\bm{\hat{y}}$|模型预测输出|
|$\bm{y}$|输入信号对应的真实输出|
|$\_i$|索引为i的值|
|$^j\_i$|第j个向量索引为i的值|
|$\bm{V}$|输入层到隐藏层的连接权矩阵|
|$\bm{b_1}$|输入层到隐藏层的偏置|
|$\bm{W}$|隐藏层到输出层的连接权矩阵|
|$\bm{b_2}$|隐藏层到输出层的偏置|
|$\bm{U}$|承接层到隐藏层的连接权矩阵|
|$f$|隐藏层激活函数|
|$g$|输出层激活函数|
|$n$|输入信号维度|
|$k$|隐藏层维度|
|$o$|输出层维度|
|$d$|训练集总数|
|$T$|总时间步|

## 正向计算

1. **输入层与承接层信号到隐藏层**
$$
\begin{equation}
    \begin{split}
        \bm{h(t)} 
        &= f(\bm{V}, \bm{U}, \bm{x(t)}, \bm{h(t-1)}, \bm{b_1}) \\
        &= f(\bm{V}\bm{x(t)} + \bm{U}\bm{h(t-1)}] + \bm{b_1})
    \end{split}
\end{equation}
$$
2. **隐藏层到输出层**
$$
\begin{equation}
    \begin{split}
        \bm{\hat{y}} 
        &= g(\bm{W}, \bm{h(t)}, \bm{b_2}) \\
        &= g(\bm{W} \bm{h(t)} + \bm{b_2})
    \end{split}
\end{equation}
$$
3. **损失计算**
* 单个训练样本的损失
Elman要求t个时刻的损失均计入总损失，在这里使用$l_2$损失，当然，随着任务类型的变换可以换为交叉熵等
$$
\begin{equation}
    \mathcal{L_j(\bm{V}, \bm{W}, \bm{U}, \bm{b_1}, \bm{b_2})} = \sum_{t=1}^T||\bm{\hat{y(t)}^j}-\bm{y(t)^j}||_2^2
\end{equation}
$$
* 训练集总损失
$$
\begin{equation}
    \mathcal{L(\bm{V}, \bm{W}, \bm{U}, \bm{b_1}, \bm{b_2})} = \sum_{j=1}^d\sum_{t=1}^T ||\bm{\hat{y(t)}^j}-\bm{y(t)^j}||_2^2
\end{equation}
$$

## 误差传递与参数更新

上述正向推理和BP网络基本一致，唯一不同的是在输入层到隐藏层的时候需要同时考虑当前时刻的输入和上一时刻隐藏层的输出。

这里值得注意的是：

上述流程虽然看起来存在环，但是实际上是不存在的。Elman依然是**有向无环**的结构。一般Elman示意图会将上一时刻和当前时刻绘制在同一张图中，但是其结构可以展开为下图的结构。显然，对于当前时刻，整个模型并不存在环，虽然误差会在不同时刻的参数中传递，但是并不会形成环。

![Elman](https://raw.githubusercontent.com/koolo233/NeuralNetworks/main/images/Elman.png "segment")

值得注意的是，Elman展开后虽然为一个有向无环图，但是其中的权重在不同时刻是**共享**的。

说回来，如果真的是有向有环图，Elman网络也不可能使用梯度下降法进行优化了

Elman网络的优化方法有两大类，分别为**实时循环学习(Real-Time Recurrent Learning, RTRL)**和**时序反向传播(BackPropagation Through Time, BPTT)**。这两类方法均基于BP算法，不同在于，RTRL在前向计算时就计算梯度并在每一个时间步更新参数，BPTT则在完成所有时间步的运算后进行梯度反向传播和参数更新。显然，BPTT需要保存大量的中间结果，其存储开销大；而RTRL则不需要存储多余的中间结果，其存储开销小，但是运算量大。

在这里，主要时对BPTT进行推导

### BPTT

* 单时间步
首先考虑仅有一个时间步，并将偏执并入权值，此时不存在沿时间的误差传递
$$
\begin{equation}
    \begin{split}
        \frac{\partial \mathcal{L}}{\partial \bm{W}} 
        &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{W}} \\
        &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{W}\bm{h(1)^j}}\frac{\partial \bm{W}\bm{h(1)^j}}{\partial \bm{W}}
    \end{split}
\end{equation}
$$
$$
\begin{equation}
    \begin{split}
        \frac{\partial \mathcal{L}_j}{\partial \bm{W}\bm{h(1)^j}} 
        &= 2(\bm{\hat{y(1)}^j}-\bm{y(1)^j})^Tdiag(g^{\prime}(\bm{W}\bm{h(1)^j})) \\
        &= \bm{\delta_1^j(1)}
    \end{split}
\end{equation}
$$
$$
\begin{equation}
    \begin{split}
    \frac{\partial \mathcal{L}}{\partial \bm{W}}
    &= \sum_{j=1}^d \bm{\delta_1^j(1)}^T\bm{h(1)^j}^T
    \end{split}
\end{equation}
$$
$$
\begin{equation}
    \begin{split}
    \frac{\partial \mathcal{L}}{\partial \bm{h(1)^j}}
    &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{W}\bm{h(1)^j}}\frac{\partial \bm{W}\bm{h(1)^j}}{\partial \bm{h(1)^j}}\\ 
    &= \sum_{j=1}^d \bm{\delta_1^j(1)}\bm{W}
    \end{split}
\end{equation}
$$
$$
\begin{equation}
    \begin{split}
        \frac{\partial \mathcal{L}}{\partial \bm{U}} 
        &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{U}} \\
        &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{U}\bm{h(0)^j}}\frac{\partial \bm{U}\bm{h(0)^j}}{\partial \bm{U}} \\
        &= \bm{0}
    \end{split}
\end{equation}
$$
$$
\begin{equation}
    \begin{split}
        \frac{\partial \mathcal{L}}{\partial \bm{V}} 
        &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{V}} \\
        &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{V}\bm{x(1)^j}}\frac{\partial \bm{V}\bm{x(1)^j}}{\partial \bm{V}}
    \end{split}
\end{equation}
$$
$$
\begin{equation}
    \begin{split}
        \frac{\partial \mathcal{L}_j}{\partial \bm{V}\bm{x(1)^j}} 
        &= \frac{\partial \mathcal{L}}{\partial \bm{h(1)^j}}\frac{\partial \bm{h(1)^j}}{\partial \bm{V}} \\
        &= \bm{\delta_1^j(1)}\bm{W}diag(f^{\prime}(\bm{V}\bm{x(1)^j})) \\
        &= \bm{\delta_2^j(1)}
    \end{split}
\end{equation}
$$
$$
\begin{equation}
    \begin{split}
    \frac{\partial \mathcal{L}}{\partial \bm{V}} 
    &= \sum_{j=1}^d \frac{\partial \mathcal{L}_j}{\partial \bm{V}\bm{x(1)^j}}\frac{\partial \bm{V}\bm{x(1)^j}}{\partial \bm{V}} \\
    &= \sum_{j=1}^d \bm{\delta_2^j(1)}^T\bm{x(1)^j}^T
    \end{split}
\end{equation}
$$

* 2时间步

经过上述分析，基本可以确定Elman网络和BP网络的梯度传递方式基本一致，唯一不同的是从输入层到隐藏层的权重$\bm{V}\notin\mathcal{R}^{k\times n}$而是$\bm{V}\in\mathcal{R}^{k\times (n+k)}$，若将偏执也纳入权重，则可以得到$\bm{V}\in\mathcal{R}^{k\times (n+k+1)}$，其余部分和BP网络完全一致。

完整的Elman网络误差传递与参数更新可以参考[BP网络误差传递与参数更新相关推导](https://github.com/koolo233/NeuralNetworks/blob/main/FullyConnectedNeuralNetwork.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import seaborn as sns
import imageio
import os
import random
from sklearn.datasets import make_blobs, make_moons

# seed
random.seed(1024)
np.random.seed(1024)
%matplotlib inline

In [2]:
# 定义单隐藏层Elman神经网络

class MyElman(object):

    def __init__(self, input_dims, hidden_dims, output_dims, lr) -> None:
        super().__init__()

        # 模型权重
        random.seed(1024)
        np.random.seed(1024)
        self.weight_input2hidden = np.random.randn(hidden_dims, input_dims+1+hidden_dims)
        self.weight_hidden2output = np.random.randn(output_dims, hidden_dims)
        
        # 激活函数
        self.activate = self.activate_func
        # 激活函数微分，事先准备好
        self.activate_dif = self.activate_dif_func

        # 中间结果
        # 输入数据
        self.input_data = None
        # 上一时刻数据
        self.last_value = np.zeros((hidden_dims, 1))
        # 真实的隐藏层输入
        self.true_hidden_input = None
        # 隐藏层输出，未激活
        self.hidden_value = None
        # 隐藏层输出，激活
        self.hidden_activate_value = None

        # 梯度结果
        self.grad_hidden2output = None
        self.grad_input2hidden = None

        # 学习率
        self.lr = lr

    def forward(self, input_data):
        """
        正向计算
        """
        
        # 扩展一个维度
        one_array = np.ones(len(input_data)).reshape(-1, 1)
        self.input_data = np.concatenate((input_data, one_array), axis=1)
        # input data : (batch, input_dims+1)

        # 拼接输入数据与上一时刻数据
        self.true_hidden_input = np.concatenate((self.input_data, self.last_value), axis=1)
        # true hidden input : (batch, input_dims+hidden_dims+1)

        # 计算隐藏层
        self.hidden_value = np.matmul(self.weight_input2hidden, self.true_hidden_input.T)
        self.hidden_activate_value = self.activate_func(self.hidden_value)
        self.last_value = self.hidden_activate_value.copy()
        # hidden_value: (hidden_dim, batch)

        # 计算输出
        output_value = np.matmul(self.weight_hidden2output, self.hidden_activate_value)
        # output_value: (output_dim, batch)

        return output_value.T
    
    def backward(self, label, pred):
        """
        梯度反向传播
        """

        # label: (batch, output_dim)
        # pred: (batch, output_dim)

        theta_1 = 2 * np.matmul(pred - label, np.eye(pred.shape[1]))
        # theta_1: (batch, output_dim)
        theta_2 = np.zeros((theta_1.shape[0], self.weight_hidden2output.shape[1]))
        for i in range(theta_1.shape[0]):
            theta_2[i, :] = np.matmul(np.matmul(theta_1[i:i+1, :], self.weight_hidden2output), 
                                      np.diag(self.activate_dif_func(self.hidden_activate_value[:, i]))).reshape(-1)

        # theta_2: (batch, hidden_dim)

        # 计算隐藏层到输出层权重的梯度
        self.grad_hidden2output = np.matmul(theta_1.T, self.hidden_value.T)
        # 计算输入层到隐藏层权重的梯度
        self.grad_input2hidden = np.matmul(theta_2.T, self.true_hidden_input)
    
    def optimize(self):
        """
        优化
        """
        self.weight_input2hidden -= self.grad_input2hidden * self.lr
        self.weight_hidden2output -= self.grad_hidden2output * self.lr


    @staticmethod
    def activate_func(x):
        """
        sigmoid
        """
        return 1/(1+np.e ** (-x))
    
    @staticmethod
    def activate_dif_func(activate_result):
        return activate_result*(1-activate_result)

In [ ]:
# 测试数据

# 时间分量
t_total = 10
# 批大小
batch_size = 5
# 输入信号维度
input_dims = 2
# 输出信号维度
output_dims = 1
# 隐藏层维度
hidden_num = 5
# 学习率
lr = 1e-3

test_input_array = np.ones((t_total, batch_size, input_dims))
test_output_array = np.random.randint(0, 2, (batch_size, output_dims))
test_model = MyElman(input_dims=input_dims,
                     hidden_dims=hidden_num, 
                     output_dims=output_dims, 
                     lr=lr)


print("before optimize")
print("input to hidden weight")
print(test_model.weight_input2hidden)
print("hidden to output weight")
print(test_model.weight_hidden2output)

for t in t_total:
    # 正向计算
    test_pred = test_model.forward(test_input_array[t])
    # 反向传播
    test_model.backward(test_output_array, test_pred)